In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [3]:
# 스타벅스코리아 홈페이지 접속
browser = webdriver.Chrome('chromedriver.exe')
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
browser.get(url)

In [4]:
# 도시별 xpath값 저장
city_xpath = []
for i in range(17):
    text = f'//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[{i+1}]/a'
    city_xpath.append(text)

In [5]:
# 도시별 이름 저장
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
city  = soup.select('ul.sido_arae_box')[0].text
city_name = []
leng = 2
for i in range(0,34,2):
    city_name_temp = city[i:leng]
    city_name.append(city_name_temp)
    leng+=2

In [6]:
starbucks_info=pd.DataFrame()

### 서울~제주 가져오기
for i in tqdm(range(len(city_name))):
    time.sleep(5)
    # 해당 지역 클릭
    browser.find_element_by_xpath(city_xpath[i]).click()
    time.sleep(5)
    # 세종시 부분 처리를 위한 if문
    if(i<16):
        # 전체 버튼 클릭
        browser.find_element_by_link_text('전체').click()
        time.sleep(3)    
    # 스타벅스 서울시 매장 리스트 HTML 파싱
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
   
    starbucks_soup_list = soup.select('li.quickResultLstCon')
    
    startbucks_list = []
    
    for item in starbucks_soup_list:
        name = item.select('strong')[0].text.strip() # 매장명
        lat = item['data-lat'].strip()
        long = item['data-long'].strip()
        store_type = item.select('i')[0].text.strip() 
        adress = str(item.select('p')[0]).split('<br/>')[0].split('>')[1]
        tel = str(item.select('p')[0]).split('<br/>')[1].split('<')[0]
        startbucks_list.append([name,lat,long,store_type,adress,tel])
    
    columns = ['매장명', '위도', '경도','매장타입','주소','전화번호']
    startbucks_df =pd.DataFrame(startbucks_list,columns=columns)
    
    starbucks_info=pd.concat([starbucks_info,startbucks_df])
    
    if(i<16):
        #지역 검색 클릭
        browser.find_element_by_xpath('//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]/h3/a').click()
browser.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [03:46<00:00, 13.33s/it]


In [7]:
starbucks_info

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,리저브 매장 2번,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,리저브 매장 2번,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,리저브 매장 2번,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,리저브 매장 2번,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,37.494668,127.062583,리저브 매장 2번,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
...,...,...,...,...,...,...
6,세종이마트,36.4712189,127.2509973,리저브 매장 2번,세종특별자치시 금송로 687(가람동 406) 이마트 세종점,1522-3232
7,세종청사,36.504565,127.270769,리저브 매장 2번,세종특별자치시 도움8로 81 (어진동),1522-3232
8,세종아름,36.51188,127.24277,리저브 매장 2번,"세종특별자치시 아름서1길 13-9 (아름동) 103,104,105,202,203호",1522-3232
9,세종소담,36.485755,127.300509,리저브 매장 2번,"세종특별자치시 소담3로 8 (소담동) 1동 12호,13호,14호,15호,15A호,16호",1522-3232


In [8]:
starbucks_info=starbucks_info.reset_index(drop=True)

In [9]:
data_copy = starbucks_info.copy()

In [10]:
data_copy = data_copy.drop(['주소','매장타입','전화번호'],axis=1)

In [11]:
data_copy.head()

,매장명,위도,경도
0,역삼아레나빌딩,37.501087,127.043069
1,논현역사거리,37.510178,127.022223
2,신사역성일빌딩,37.514132,127.020563
3,국기원사거리,37.499517,127.031495
4,스탈릿대치R,37.494668,127.062583


In [12]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1571 entries, 0 to 1570
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     1571 non-null   object
 1   위도      1571 non-null   object
 2   경도      1571 non-null   object
dtypes: object(3)
memory usage: 36.9+ KB


In [13]:
data_copy=data_copy.dropna()

In [14]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1571 entries, 0 to 1570
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     1571 non-null   object
 1   위도      1571 non-null   object
 2   경도      1571 non-null   object
dtypes: object(3)
memory usage: 49.1+ KB


In [15]:
data_copy['위도']=data_copy['위도'].astype(float)
data_copy['경도']=data_copy['경도'].astype(float)

In [16]:
import requests

def getCount(id, lat, lng, category):
    #radius는 미터단위 여기선 1km 반경 이내 값을 도출
    url = "https://dapi.kakao.com/v2/local/search/category.json?x=" + lng + "&y=" + lat + "&radius=1000&category_group_code="+category 
    apikey = "5e5acc55c8584f36abffe26633c82550"
    r = requests.get( url, headers={'Authorization' : 'KakaoAK ' + apikey })
    return r.json()["meta"]["total_count"]

In [17]:
result = [['매장명', '대형마트', '편의점', '어린이집,유치원', '학교', '학원',
          '주차장', '주유소,충전소', '지하철역', '은행', '문화시설', '중개업소',
          '공공기관', '관광명소', '숙박', '음식점', '카페', '병원', '약국']]
category_code = ['MT1','CS2', 'PS3', 'SC4','AC5',
                 'PK6','OL7','SW8','BK9','CT1','AG2',
                'PO3','AT4','AD5','FD6','CE7','HP8','PM9']

# MT1 대형마트
# CS2 편의점
# PS3 어린이집, 유치원
# SC4 학교
# AC5 학원
# PK6 주차장
# OL7 주유소, 충전소
# SW8 지하철역
# BK9 은행
# CT1 문화시설
# AG2 중개업소
# PO3 공공기관
# AT4 관광명소
# AD5 숙박
# FD6 음식점
# CE7 카페
# HP8 병원
# PM9 약국

for v in tqdm(data_copy.iterrows(), total=data_copy.shape[0]):
    id = str(v[1]['매장명'])
    lat = str(v[1]['위도'])
    lng = str(v[1]['경도'])
    item_result = [id]
    for cate in category_code:
        item_result.append(getCount(id, lat, lng, cate))
    result.append(item_result)

100%|██████████████████████████████████████████████████████████████████████████████| 1571/1571 [54:07<00:00,  2.07s/it]


In [18]:
print(result[0])

['매장명', '대형마트', '편의점', '어린이집,유치원', '학교', '학원', '주차장', '주유소,충전소', '지하철역', '은행', '문화시설', '중개업소', '공공기관', '관광명소', '숙박', '음식점', '카페', '병원', '약국']


In [19]:
arr = np.array(result)
df = pd.DataFrame(arr)

In [20]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,매장명,대형마트,편의점,"어린이집,유치원",학교,학원,주차장,"주유소,충전소",지하철역,은행,문화시설,중개업소,공공기관,관광명소,숙박,음식점,카페,병원,약국
1,역삼아레나빌딩,3,179,23,5,572,220,5,3,186,26,596,7,0,84,2182,580,312,78
2,논현역사거리,5,145,27,3,353,219,5,6,117,25,360,11,2,52,2190,540,599,80
3,신사역성일빌딩,4,112,25,3,340,185,5,3,104,37,333,8,4,34,1945,523,432,62
4,국기원사거리,6,212,21,4,745,336,4,5,204,37,618,9,3,105,3048,840,754,125


In [21]:
df.to_csv('스타벅스 주변시설 갯수.csv') 

In [22]:
starbucks_info.to_csv('스타벅스 매장 정보.csv') 

In [ ]:
star=pd.read_csv('스타벅스_주위환경.csv',encoding='CP949')